# Fashion MNIST Classification Using CNN in Keras

In [33]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np

## Create Callbacks Class

In [47]:
class EarlyStoppingAtMinLoss(tf.keras.callbacks.Callback):
    """Stop training when the loss is at its min, i.e. the loss stops decreasing.
    
    Arguments:
        patience: Number of epochs to wait after min has been hit. After this
        number of no impprovement, training stops.
    """
    def __init__(self, patience=0):
        super().__init__()
        
        self.patience = patience
        
        # best_weights to store the weights at which the minimum loss occurs.
        self.best_weights = None
        
    def on_train_begin(self, logs=None):
        # The number of epoch it has waited when loss is no longer minumum.
        self.wait = 0
        # The epoch the training stops at.
        self.stopped_epoch = 0
        # Initialize the best as infinity.
        self.best = np.Inf
        
    def on_epoch_end(self, epoch, logs=None):
        current = logs.get('loss')
        if np.less(current, self.best):
            self.best = current
            self.wait = 0
            # Record the best weights if current results is better (less).
            self.best_weights = self.model.get_weights()
        else:
            self.wait += 1
            if self.wait >= self.patience:
                self.stopped_epoch = epoch
                self.model.stop_training = True
                print('\nRestoring model weights from the end of the best epoch.')
                self.model.set_weights(self.best_weights)
    
    def on_train_end(self, logs=None):
        if self.stopped_epoch > 0:
            print(f'Epoch {self.stopped_epoch + 1}: early stopping')

## CNN Architecture

In [45]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images, test_images = train_images.reshape(60000, 28, 28, 1), test_images.reshape(10000, 28, 28, 1)
train_images, test_images = train_images / 255, test_images / 255
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 1600)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 128)               204928    
_________________________________________________________________
dense_19 (Dense)             (None, 10)               

## Training and Evaluate Model

In [46]:
model.fit(train_images, train_labels, epochs=30, callbacks=[EarlyStoppingAtMinLoss()])
test_loss = model.evaluate(test_images, test_labels)

Train on 60000 samples
Epoch 1/30
60000/60000 [==============================] - 7s 111us/sample - loss: 0.4207 - accuracy: 0.8468
Epoch 2/30
60000/60000 [==============================] - 6s 105us/sample - loss: 0.2822 - accuracy: 0.8955
Epoch 3/30
60000/60000 [==============================] - 6s 107us/sample - loss: 0.2367 - accuracy: 0.9121
Epoch 4/30
60000/60000 [==============================] - 7s 110us/sample - loss: 0.2035 - accuracy: 0.9235
Epoch 5/30
60000/60000 [==============================] - 7s 110us/sample - loss: 0.1732 - accuracy: 0.9348
Epoch 6/30
60000/60000 [==============================] - 7s 110us/sample - loss: 0.1483 - accuracy: 0.9428
Epoch 7/30
60000/60000 [==============================] - 7s 110us/sample - loss: 0.1261 - accuracy: 0.9525
Epoch 8/30
60000/60000 [==============================] - 6s 108us/sample - loss: 0.1067 - accuracy: 0.9595
Epoch 9/30
60000/60000 [==============================] - 7s 110us/sample - loss: 0.0921 - accuracy: 0.9654
Epoch